In [1]:
import sys
import os
os.chdir("..")
sys.path.append(os.getcwd() + '/src')

from elisa.conf import config
config.LOG_CONFIG = os.getcwd() + 'jupyter_tutorials/logging.json'

/home/miro/ELISa/src/elisa/conf/config.py:161: UserWarning: path /home/miro/ELISa/atmosphere/ck93
to kurucz 1993 atmosphere atlas doesn't exists
Specifiy it in elisa_conf.ini file
  "Specifiy it in elisa_conf.ini file".format(K93_ATM_TABLES))


# Demo No. 4 - Spots

This demo will demonstrate how to generate spots on the surface of the given component.

## Generating spot metadata

Before the standard procedure of defining the components and the binary system itself, lets first specify the parameters of our spots.

In [1]:
spots_metadata = {
    "primary":
        [
            #  Spot 1
            {"longitude": 0,
             "latitude": 45,
             # "angular_density": 1,
             "angular_radius": 27,
             "temperature_factor": 1.01},
            #  Spot 2
            {"longitude": 30,
             "latitude": 30,
             # "angular_density": 2,
             "angular_radius": 15,
             "temperature_factor": 0.99},
            #  Spot 3
            {"longitude": 40,
             "latitude": 50,
             # "angular_density": 2,
             "angular_radius": 15,
             "temperature_factor": 1.02},
        ],

    "secondary":
        [
            #  Spot 1
            {"longitude": 0,
             "latitude": 45,
             # "angular_density": 3,
             "angular_radius": 14,
             "temperature_factor": 1.02},
            #  Spot 2
            {"longitude": 30,
             "latitude": 65,
             # "angular_density": 3,
             "angular_radius": 23,
             "temperature_factor": 0.995},
        ]
}

where spots on given component are defined by a list of dictionaries for each spot. Each spot is defined by 4 parameters `longitude`, `latitude`, `angular_radius` and `temperature_factor`. Angular density of the surfa